## Preprocessing

In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=['EIN','NAME'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [7]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

There are three columns that have more than 10 unique values. The categorical types will need to be binned to reduce the number of unique values. Numerical data does not need to be binned.

APPLICATION_TYPE: 17

CLASSIFICATION: 71

ASK_AMT: 8,747

In [8]:
# Look at APPLICATION_TYPE value counts for binning
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# if type < 100, group as 'other'
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning

cls1 = df['CLASSIFICATION'].value_counts()
cls1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2500        1
C6100        1
C2380        1
C1248        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1

cls = df['CLASSIFICATION'].value_counts().loc[lambda x : x>1] 
print(len(cls))
cls

45


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1256        2
C1246        2
C3200        2
C1267        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# FILTER CLS by values less than 1000. Use index for list
classifications_to_replace = (cls1[cls1<1000]).index
classifications_to_replace

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C6000', 'C1800', 'C1250', 'C8200', 'C1278', 'C1238',
       'C1237', 'C1235', 'C7210', 'C1720', 'C4100', 'C2400', 'C1600', 'C1257',
       'C2710', 'C1260', 'C0', 'C1256', 'C1246', 'C3200', 'C1267', 'C1234',
       'C5200', 'C4200', 'C1236', 'C1245', 'C2600', 'C4120', 'C3700', 'C8210',
       'C2170', 'C1820', 'C2190', 'C1900', 'C1283', 'C4500', 'C1732', 'C1370',
       'C1570', 'C1580', 'C1728', 'C2570', 'C2561', 'C2500', 'C6100', 'C2380',
       'C1248', 'C2150'],
      dtype='object')

In [13]:
# Replace in dataframe
for j in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(j,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`

print(df.dtypes)
print('Status:')
print(df['STATUS'].value_counts())
print('INCOME_AMT')
print(df['INCOME_AMT'].value_counts())
print('Is successful:')
print(df['IS_SUCCESSFUL'].value_counts())
df

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object
Status:
1    34294
0        5
Name: STATUS, dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64
Is successful:
1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


The STATUS data is imbalanced - 5 out of 34294 instances are negative. This may be something to consider later when optimizing the model.


In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
df_cat = pd.get_dummies(df)
df_cat

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# Split our preprocessed data into our features and target arrays

# The target column is 'IS_SUCCESSFUL'
X = df_cat.drop(['IS_SUCCESSFUL'], axis=1)
y = df_cat['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = (train_test_split(X, y, random_state=1))

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer - INPUT FEATURES: 44. 2 neurons per feature: 88
nn_model.add(tf.keras.layers.Dense(units=88, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 88)                3960      
                                                                 
 dense_1 (Dense)             (None, 30)                2670      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,661
Trainable params: 6,661
Non-trainable params: 0
_________________________________________________________________


# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=18, activation="relu", input_dim=9))

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer


# Check the structure of the model
nn.summary()

In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [20]:
# Train the model
# Create a callback that saves the model's weights every five epochs.
model_weights = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model01/checkpoint',
    verbose= 1,
    save_weights_only= True,
    mode='auto',
    save_freq='epoch',
    period=5
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[model_weights])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5725 - accuracy: 0.7182
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5561 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5511 - accuracy: 0.7300
Epoch 5/100
789/804 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7306
Epoch 5: saving model to ./model01\checkpoint
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 2s 2ms

804/804 [==============================] - 1s 1ms/step - loss: 0.5361 - accuracy: 0.7391
Epoch 65/100
798/804 [============================>.] - ETA: 0s - loss: 0.5360 - accuracy: 0.7398
Epoch 65: saving model to ./model01\checkpoint
804/804 [==============================] - 1s 2ms/step - loss: 0.5363 - accuracy: 0.7393
Epoch 66/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5363 - accuracy: 0.7394
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7393
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7399
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5357 - accuracy: 0.7402
Epoch 70/100
788/804 [============================>.] - ETA: 0s - loss: 0.5355 - accuracy: 0.7393
Epoch 70: saving model to ./model01\checkpoint
804/804 [==============================] - 2s 3ms/step - loss: 0.5352 - accuracy: 0.7391
Epoch 71/100
804/804 [=============

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5602 - accuracy: 0.7294 - 417ms/epoch - 2ms/step
Loss: 0.5602009892463684, Accuracy: 0.7294460535049438


In [22]:
# STARTER CODE
# Evaluate the model using the test data

# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [23]:
# Export our model to HDF5 file

nn_model.save('AlphabetSoupCharity.h5')

### Explore Model Optimization ###

#### Create Model 2 using best accuracy weights ####

In [27]:
# Train the model
# Create a callback that saves the model's weights every epoch IF it is the best seen so far.
best_model_weights = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model02/checkpoint',
    verbose= 1,
    save_weights_only= True,
    monitor= 'accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch'
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[best_model_weights])

Epoch 1/100
778/804 [============================>.] - ETA: 0s - loss: 0.5308 - accuracy: 0.7420
Epoch 1: accuracy improved from -inf to 0.74223, saving model to ./model02\checkpoint
804/804 [==============================] - 1s 2ms/step - loss: 0.5306 - accuracy: 0.7422
Epoch 2/100
786/804 [============================>.] - ETA: 0s - loss: 0.5322 - accuracy: 0.7418
Epoch 2: accuracy did not improve from 0.74223
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7420
Epoch 3/100
793/804 [============================>.] - ETA: 0s - loss: 0.5313 - accuracy: 0.7418
Epoch 3: accuracy did not improve from 0.74223
804/804 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 0.7416
Epoch 4/100
769/804 [===========================>..] - ETA: 0s - loss: 0.5304 - accuracy: 0.7426
Epoch 4: accuracy did not improve from 0.74223
804/804 [==============================] - 1s 1ms/step - loss: 0.5305 - accuracy: 0.7420
Epoch 5/100
783/804 [========

804/804 [==============================] - 1s 2ms/step - loss: 0.5306 - accuracy: 0.7416
Epoch 36/100
798/804 [============================>.] - ETA: 0s - loss: 0.5301 - accuracy: 0.7426
Epoch 36: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 2ms/step - loss: 0.5302 - accuracy: 0.7423
Epoch 37/100
803/804 [============================>.] - ETA: 0s - loss: 0.5303 - accuracy: 0.7421
Epoch 37: accuracy did not improve from 0.74285
804/804 [==============================] - 2s 3ms/step - loss: 0.5302 - accuracy: 0.7421
Epoch 38/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5301 - accuracy: 0.7422
Epoch 38: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 1ms/step - loss: 0.5303 - accuracy: 0.7423
Epoch 39/100
799/804 [============================>.] - ETA: 0s - loss: 0.5309 - accuracy: 0.7413
Epoch 39: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 1ms/step - los

804/804 [==============================] - 2s 3ms/step - loss: 0.5298 - accuracy: 0.7421
Epoch 71/100
778/804 [============================>.] - ETA: 0s - loss: 0.5303 - accuracy: 0.7417
Epoch 71: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 1ms/step - loss: 0.5300 - accuracy: 0.7419
Epoch 72/100
788/804 [============================>.] - ETA: 0s - loss: 0.5299 - accuracy: 0.7418
Epoch 72: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 2ms/step - loss: 0.5299 - accuracy: 0.7416
Epoch 73/100
798/804 [============================>.] - ETA: 0s - loss: 0.5299 - accuracy: 0.7423
Epoch 73: accuracy did not improve from 0.74285
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7421
Epoch 74/100
771/804 [===========================>..] - ETA: 0s - loss: 0.5295 - accuracy: 0.7414
Epoch 74: accuracy did not improve from 0.74285
804/804 [==============================] - 1s 1ms/step - los

In [28]:
nn_model.load_weights('./model02/checkpoint')
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 2 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5782 - accuracy: 0.7298 - 275ms/epoch - 1ms/step
Model 2 Loss: 0.578238844871521, Accuracy: 0.7297959327697754


##### Model 2 performed slightly worse than Model 1. #### 
The Accuracy score is the same and the Loss score is slightly greater.

#### Optimise using Keras-tuner ###

In [29]:
!pip install keras-tuner

In [30]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=88,
        max_value=132,
        step=4), activation=activation, input_dim=44))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=80,
            step=4),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [31]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [32]:
 # Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 25 Complete [00h 00m 33s]
val_accuracy: 0.7300291657447815

Best val_accuracy So Far: 0.7335277199745178
Total elapsed time: 00h 07m 28s

Search: Running Trial #26

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
88                |92                |first_units
3                 |5                 |num_layers
45                |53                |units_0
33                |53                |units_1
21                |33                |units_2
1                 |29                |units_3
29                |33                |units_4
13                |49                |units_5
20                |3                 |tuner/epochs
7                 |0                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |0                 |tuner/round
0020              |None              |tuner/trial_id

Epoch 8/20
804/804 [==============================] - 5s 5ms/step - loss: 0.5749 - accu

KeyboardInterrupt: 